In [2]:
import csv
import pandas as pd
import re
from konlpy.tag import Kkma

nlpy = Kkma()
file = open('hashcode_classification2020_train.csv', 'r', encoding='utf-8')
line = csv.reader(file)

f = open("goodWordsk.txt", 'w')

english=open("English_Dict.txt", 'r')
korean=open("Korean_Dict.txt", 'r')

englishDict=[]
sentences = english.readlines()
for sentence in sentences: #한 문장씩
    englishDict.append(sentence[:-1])

koreanDict=[]
sentences = korean.readlines()
for sentence in sentences: #한 문장씩
    koreanDict.append(sentence[:-1])


for line_text in line:
    eng=re.sub('[^a-zA-Z]',' ',line_text[0]+line_text[1]).strip()
    eng_temp=eng.split(' ')
    eng_temp = list(filter(None, eng_temp))
    for i in range(len(eng_temp)):
        if eng_temp[i] not in englishDict:
            eng_temp[i]=''
    eng_temp = list(filter(None, eng_temp))
    
    kor=nlpy.nouns(line_text[0]+line_text[1])
    for i in range(len(kor)):
        if kor[i] not in koreanDict:
            kor[i]=''
    kor = list(filter(None, kor))
    
    
    s=' '.join(eng_temp)+' '+' '.join(kor)+','+line_text[2]
    f.write(s)
    f.write('\n')
    
    
#     kor=
file.close()
f.close()
english.close()
korean.close()

NameError: name 'special' is not defined

In [21]:
import pandas
import csv
file= open('goodWordsk.txt', 'r')
line = file.readline()
training_data=[]
while line:
    data=[line[:-3],line[-2:-1]]
    training_data.append({'data': data[0], 'flag': data[1]})
    line=file.readline()
print(training_data[0])

training_data=pandas.DataFrame(training_data, columns=['data', 'flag'])
training_data.to_csv("train_data.csv", sep=',', encoding='utf-8')
print(training_data.data.shape)
file.close()

{'data': 'wait sleep wait sleep ', 'flag': '3'}
(2592,)


In [22]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [24]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#clf = MultinomialNB().fit(X_train_tfidf, training_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.01, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [25]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

docs_new = "numpy as np print array lst import coding arr utf"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

print(predicted[0])

5


In [26]:
predicted = loaded_model.predict(X_test)
result_bayes = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_bayes.to_csv('res_bayes.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(predicted_item, ' - ', result)

3  -  3
5  -  5
3  -  3
4  -  4
1  -  1
5  -  5
3  -  3
4  -  4
5  -  5
5  -  5
3  -  3
5  -  5
3  -  3
3  -  3
4  -  4
3  -  3
4  -  4
2  -  2
1  -  1
5  -  5
4  -  4
5  -  5
3  -  3
1  -  1
5  -  5
3  -  3


In [27]:
from sklearn.metrics import confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)

[[3 0 0 0 0]
 [0 1 0 0 0]
 [0 0 9 0 0]
 [0 0 0 5 0]
 [0 0 0 0 8]]


In [28]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.01, random_state=42)

clf_neural.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [29]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

In [30]:
predicted = clf_neural.predict(X_test)
result_softmax = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_softmax.to_csv('res_softmax.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(predicted_item, ' - ', result)

3  -  3
5  -  5
3  -  3
4  -  4
2  -  1
5  -  5
3  -  3
4  -  4
5  -  5
5  -  5
3  -  3
5  -  5
3  -  3
3  -  3
4  -  4
3  -  3
4  -  4
2  -  2
1  -  1
5  -  5
4  -  4
5  -  5
3  -  3
1  -  1
5  -  5
3  -  3


In [31]:
from sklearn import svm
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.01, random_state=42)
clf_svm.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

In [32]:
predicted = clf_svm.predict(X_test)
result_svm = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')
for predicted_item, result in zip(predicted, y_test):
    print(predicted_item, ' - ', result)

3  -  3
5  -  5
3  -  3
4  -  4
1  -  1
5  -  5
3  -  3
4  -  4
5  -  5
5  -  5
3  -  3
5  -  5
3  -  3
3  -  3
4  -  4
3  -  3
4  -  4
2  -  2
1  -  1
5  -  5
4  -  4
5  -  5
3  -  3
1  -  1
5  -  5
3  -  3


In [33]:
docs_new=[]
f=open("testwords.txt",'r')
sentences = f.readlines()
for sentence in sentences:
    docs_new.append([sentence[:-1]])
f.close()
f2 = open('output2.csv', 'w', newline='')
wr = csv.writer(f2)
wr.writerow(['label'])
for docs in docs_new:
    X_new_counts = loaded_vec.transform(docs)
    X_new_tfidf = loaded_tfidf.transform(X_new_counts)
    predicted = clf_svm.predict(X_new_tfidf)
    wr.writerow([predicted[0]])

f2.close()

In [34]:
docs_new=[]
f=open("testwords.txt",'r')
sentences = f.readlines()
for sentence in sentences:
    docs_new.append([sentence[:-1]])
f.close()
f2 = open('output2.csv', 'w', newline='')
wr = csv.writer(f2)
wr.writerow(['label'])
for docs in docs_new:
    X_new_counts = loaded_vec.transform(docs)
    X_new_tfidf = loaded_tfidf.transform(X_new_counts)
    predicted = clf_neural.predict(X_new_tfidf)
    wr.writerow([predicted[0]])

f2.close()

In [35]:
docs_new=[]
f=open("testwords.txt",'r')
sentences = f.readlines()
for sentence in sentences:
    docs_new.append([sentence[:-1]])
f.close()
f2 = open('output2.csv', 'w', newline='')
wr = csv.writer(f2)
wr.writerow(['label'])
for docs in docs_new:
    X_new_counts = loaded_vec.transform(docs)
    X_new_tfidf = loaded_tfidf.transform(X_new_counts)
    predicted = loaded_model.predict(X_new_tfidf)
    wr.writerow([predicted[0]])

f2.close()